In [ ]:
%matplotlib inline
import copy
import numpy as np
import pandas as pd
import networkx as nx
import csv, pickle
from datetime import timedelta, datetime
import itertools
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm import tqdm_notebook as tqdm
import random
import pickle
import ast
import sys, os
from pathlib import Path
p = Path.cwd()
path = p.parent.parent.parent
sys.path.append(str(path))
from src import utils
from src import conflict
from src import voyage
from src import fda_wrapper as fjw
from pyproj import Geod

date = datetime.now()
stamp = '{0:%Y_%m_%d}'.format(date)
_folder = str(p) + '/result/run_' + stamp + '/'
if os.path.exists(_folder):
    pass
else:
    os.makedirs(_folder)

In [ ]:
_folder

In [ ]:
import dimod
from pyqubo import Array, Constraint, Placeholder, solve_qubo, Sum, Binary
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave

In [ ]:
# 実行するときだけコメントアウト外す
endpoint = 'https://cloud.dwavesys.com/sapi'
token = 'IPA-62f138a24969d85d910780e46165c1189d984ce5'
solverDW = 'DW_2000Q_5' # 'DW_2000Q_2_1' 'DW_2000Q_VFYC_2_1' 'DW_2000Q_VFYC_5', 'DW_2000Q_5'
sampler = EmbeddingComposite(DWaveSampler(endpoint=endpoint, token=token, solver=solverDW))

In [ ]:
def calc_anneal_schedule(sp, tp):
    time = [0.0, sp, sp + tp, 2*sp + tp] # unit in [us]
    flux = [0.0, 0.5, 0.5, 1.0]
    if sp == 0:
        print('invalid sp!!')
        sys.exit(1)
    ar = len(flux)
    schedule = [[time[i], flux[i]] for i in range(ar)]
    return schedule

# make QUBO
def make_hami_model(Nf, Nd, klist):
    x = Array.create('x', (Nf, Nd), 'BINARY')

    f_encoding = 0
    for i in range(Nf):
        f_encoding += Constraint((Sum(0, Nd, lambda j: x[i, j]) - 1)**2, label='encoding_{}'.format(i))

    f_delay = 0
    for i in range(Nf):
        f_delay += Constraint(Dd*Sum(0, Nd, lambda j: j*x[i, j]), label='delay_{}'.format(i))

    f_conflict = 0
    for k in range(len(klist)):
        Ck = klist[k]
    #     print(len(Ck.conflictsteps))
        i = int(Ck.voyages[0].id)
        j = int(Ck.voyages[1].id)
        Bk = klist[k][0].get_bk(time_separation)
        fk = 0
        for p in range(Nd):
            for q in range(Nd):
                Dk = Dd*(p-q)
                if Dk >= Bk[0] and Dk <= Bk[1]:
                    fk += x[i][p]*x[j][q]
                else:
                    pass
        if fk != 0:
            f_conflict += Constraint(fk, label='conflict_{}'.format(k))

    # Define hamiltonian H
    L_enc = Placeholder('L_enc')
    L_del = Placeholder('L_del')
    L_cof = Placeholder('L_cof')

    H = L_enc * f_encoding + L_del*f_delay + L_cof * f_conflict
    # Compile model
    model = H.compile()
    return model

import re
def PyQdic_to_DwaveQubo(qdic, offset, shape):           
    # pyqubo x0, x1 to Fujitsu DA BinaryPolynomial
    # "shape" should be same as "(N,K) in Array.create('x', (N, K), 'BINARY')"
    dwdic = {}
    N, M = shape # row, col
    for k in qdic:
        x0, x1 = k
        s = list(map(lambda x: int(x[1:][:-1]), re.findall('\[\d+\]', x0)))
        t = list(map(lambda x: int(x[1:][:-1]), re.findall('\[\d+\]', x1)))
        ns = M * s[0] + s[1]
        nt = M * t[0] + t[1]
#         print(k, ns, nt)
        dwdic[(ns, nt)] = qdic[k]
    return dwdic

def DwaveSol_to_PyqSol(sol, shape):
    N, M = shape # row, col
    PyQ_res = dict()
    for n in range(N):
        for m in range(M):
            i = M * n + m
            PyQ_res['x[{}][{}]'.format(n, m)] = int(sol[i])
    return PyQ_res

In [ ]:
flag_chain_fix = True

Nf = 6
Nd = 8
if flag_chain_fix:
    sampler = DWaveSampler(token=token, solver=solverDW)
    para = sampler.properties.get('parameters', {})
    n = Nf * Nd
    M = para.get('M',16)
    N = para.get('N',16)
    L = para.get('L', 4)
    _hardware_adj = sampler.adjacency # ハードウェアの結合度を取得(使えないカップラーやQUBITはこの中に含まれない)
    el = sampler.edgelist
    nodes = sampler.nodelist
    # -------------- find_clique_embedding を使う方法--------------
    # chainの長さの揃った必要QUBIT数に応じた完全グラフを探してくれる。シンプル。速い。
    _embedding = dwave.embedding.chimera.find_clique_embedding(n, 16, 16, 4, target_edges=el)
chain_st = 3.2

# Execute experiment : data 6x8 (DW)
---

In [ ]:
Dds = ['030'] # ['010', '030', '050']
INCs = ['_1', '_2', '_3']

In [ ]:
date_DW = datetime.now()
stamp_DW = '{0:%H%M%S}'.format(date_DW)
folder = _folder + '/attime_' + stamp_DW + '/'
if os.path.exists(folder):
    pass
else:
    os.makedirs(folder)
    
_n_ = 15
_nSol = 100
sp = 120
tp = 220
L_enc_ratio = np.linspace(0.1, 6, _n_)
L_cof_ratio = np.linspace(0.1, 3, _n_)

exp_result = dict()
exp_result['solver'] = solverDW
exp_result['Le'] = L_enc_ratio
exp_result['Lc'] = L_cof_ratio
exp_result['Nsol'] = _nSol
exp_result['Chain'] = chain_st
exp_result['sp, tp'] = [sp, tp]

schedule = calc_anneal_schedule(sp, tp)
for dd in tqdm(Dds):
    _res_dic = dict()
    __res_dic = dict()
    for inc in tqdm(INCs):
        exp_path = 'C:\\Users\\sho60\\OneDrive\\Documents\\github\\annealing_for_ships\\\
data\\csv\\6x8\\csv_6x8_Dd030' + dd + '\\20191125' + inc
        exppath = Path(exp_path)
        exp_m = utils.ExpManager(None, exppath)
        klist = exp_m.read_klist()
        print(len(klist))
        Nf = len(exp_m.param['v_ids'])
        Nd = exp_m.param['Nd']
        Dd = exp_m.param['Dd'] # hour
        voyages = Nf

        separation = exp_m.param['sep'] # km
        time_separation = exp_m.param['t_sep'] # h
        
        model = make_hami_model(Nf, Nd, klist)
        __res_dic['model'] = model
        res_list_DW = list()
        for p in tqdm(range(len(L_enc_ratio))):
            res_list = list()
            for q in tqdm(range(len(L_cof_ratio))):
                feed_dict = {'L_enc': L_enc_ratio[p], 'L_del':0.1, 'L_cof':L_cof_ratio[q]}
                qubo, offset = model.to_qubo(feed_dict=feed_dict)
                dwQubo = PyQdic_to_DwaveQubo(qubo, offset, (Nf, Nd))
                embQubo = dwave.embedding.embed_qubo(dwQubo, _embedding, _hardware_adj, chain_strength=chain_st)
                flag = 0
                while flag == 0:
                    try:
                        _response = sampler.sample_qubo(embQubo, num_reads=_nSol,
                                                    readout_thermalization=100,
                                                    programming_thermalization=100,
                                                    anneal_schedule=schedule)
                        bqm = dimod.BinaryQuadraticModel.from_qubo(dwQubo, offset=offset)
                        response = dwave.embedding.unembed_sampleset(_response, _embedding, bqm,
                                                                     chain_break_method=None)
                        if len(response.record) == _nSol:
                            flag = 1
                    except:
                        pass
                res_list.append(response)
            res_list_DW.append(res_list)
        __res_dic['result'] = copy.deepcopy(res_list_DW)
        __res_dic['params'] = {'Nf': Nf, 'Nd': Nd, 'Dd': Dd, 'S_sep': separation,'T_sep': time_separation}
        _res_dic[inc] = copy.deepcopy(__res_dic)
    exp_result[dd] = _res_dic

In [ ]:
f = open(folder + 'expres_DW_{}x{}_'.format(Nf, Nd) + stamp_DW + '.expres','wb')
pickle.dump(exp_result, f)
f.close()

In [ ]:
stamp_DW